In [15]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
from src.dataloaders import PointOnlyDataloader, NoiseDataLoader
from src.solutions import EllipseDataFitting

dataset_dir = Path('../dataset')
root_file = "farichsim_pi-pi-_45-360deg_1200.0k_ideal_2020-12-24_rndm.root"
root_path = dataset_dir / root_file

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
df = pd.read_csv('res.csv')

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [4]:
df.memory_usage()

Index              128
entry        252263168
subentry     252263168
x_c          252263168
y_c          252263168
z_c          252263168
t_c0         252263168
0            252263168
100          252263168
55           252263168
201.05       252263168
1.0519133    252263168
dtype: int64

In [8]:
dl = PointOnlyDataloader(root_path, verbose=False)
alg = EllipseDataFitting(save_graphics=True)



In [16]:
dl = NoiseDataLoader(root_path, verbose=False, noise_freq_per_sqmm=2e5)
max_delta = 0
for df in tqdm(dl.genChunkFromRoot(event_chunk_size=20000), total=dl.get_total_cnt()/20000):
    df.to_csv('dataset_with_noise_2e5.csv', mode='a', header=None)
    break
    # max_delta = max(df['t_c'].max() - df['t_c'].min(), max_delta)

  0%|          | 0/60.0 [00:00<?, ?it/s]


TypeError: super(type, obj): obj must be an instance or subtype of type

In [22]:
 max_delta

2.3193755

In [3]:
centers, vectors, normals, point1, point2, point3 = alg.run(df)


  4%|▍         | 2/47 [00:38<14:24, 19.22s/it]


In [4]:
centers = pd.DataFrame(np.array(centers), columns=['x_center','y_center', 't_center'])

In [5]:
centers

,x_center,y_center,t_center
0,-50.042775,-14.714778,0.721234


In [8]:
normals = pd.DataFrame(np.array(normals), columns=['x_normal','y_normal', 't_normal'])

In [9]:
points1 = pd.DataFrame(np.array(point1), columns=['x_p1','y_p1', 't_p1'])

In [10]:
points1

,x_p1,y_p1,t_p1
0,-105.279999,-29.76,0.766782


In [11]:
df

x_c         y_c         z_c       t_c
entry subentry                                              
5     0        -351.040009  399.679993  201.050003  2.387565
      1         -85.120003  -80.959999  201.050003  0.870651
      2        -105.279999  -29.760000  201.050003  0.766782
      3        -105.279999  -25.600000  201.050003  0.767961
      4        -108.639999   -8.800000  201.050003  0.767712
      5         -98.559998   15.520000  201.050003  0.754884
      6        -101.919998    5.440000  201.050003  0.756660
      7        -105.279999    5.440000  201.050003  0.761616
      8         -98.559998   18.879999  201.050003  0.753252
      9         -88.480003   29.760000  201.050003  0.745859
      10        -70.879997 -178.240005  201.050003  1.189885
      11        -64.160004  -67.519997  201.050003  0.743735
      12        -74.239998  -64.160004  201.050003  0.750375
      13        -80.959999  -64.160004  201.050003  0.759340
      14        -67.519997  -57.439999  201.050003  0.740059
      15        -57.439999   22.240000  201.050003  1.134990
      16        -74.239998   33.119999  201.050003  0.730597
      17        -77.599998   36.480000  201.050003  0.732978
      18        -74.239998   36.480000  201.050003  0.730177
      19        -70.879997   36.480000  201.050003  0.725091
      20        -74.239998  219.360001  201.050003  1.580266
      21        -43.200001  -67.519997  201.050003  0.729207
      22        -53.279999  -70.879997  201.050003  0.736687
      23        -33.119999  -67.519997  201.050003  0.721118
      24        -53.279999  -70.879997  201.050003  0.738963
      25        -43.200001   43.200001  201.050003  0.706602
      26        -49.919998   43.200001  201.050003  0.709753
      27        -49.919998   39.840000  201.050003  0.708978
      28        -12.160000 -205.919998  201.050003  1.162304
      29        -22.240000  -60.799999  201.050003  0.709096
      30         -2.080000  -46.560001  201.050003  0.693336
      31         -5.440000  -49.919998  201.050003  0.695881
      32         -5.440000   22.240000  201.050003  0.679680
      33         -5.440000   18.879999  201.050003  0.679365
      34        -22.240000   33.119999  201.050003  0.689608
      35          8.800000  -22.240000  201.050003  0.680270
      36          8.800000    2.080000  201.050003  0.677087
      37          2.080000    8.800000  201.050003  0.677228
      38          5.440000    8.800000  201.050003  0.677986
      39          2.080000    8.800000  201.050003  0.676161
      40         15.520000   88.480003  201.050003  0.832270
      41         46.560001 -108.639999  201.050003  0.802872
      42         43.200001  -64.160004  201.050003  0.735939
      43         36.480000  -57.439999  201.050003  0.738325
      44         29.760000   80.959999  201.050003  0.886469
      45        122.879997  -60.799999  201.050003  0.866539
      46        164.000000 -223.520004  201.050003  1.455103
      47        153.919998  -22.240000  201.050003  0.902340
      48        188.320007  -74.239998  201.050003  0.973741

In [6]:
vec_1 = pd.DataFrame(np.array([i[0] for i in vectors]), columns=['x_v_1','y_v_1', 't_v_1'])

In [7]:
vec_1

,x_v_1,y_v_1,t_v_1
0,-105.870569,-27.304654,0.766708


In [14]:
point1

[Point([-105.28     ,  -29.76     ,    0.7667816], dtype=float32)]

In [1]:
ax = dl.plot_solution(df)

for norm, point11, point22, point33 in zip(normals, point1, point2, point3):
    # print(norm, point)
    # norm = norm / np.linalg.norm(norm)
    ax.plot([vec_1['t_v_1'][0], centers['t_center'][0]],
             [ vec_1['x_v_1'][0], centers['x_center'][0]],
             [vec_1['x_v_1'][0], centers['y_center'][0]],
             label='Vec_1')
    ax.scatter3D(centers['t_center'][0], centers['x_center'][0], centers['y_center'][0], 'g.', label='Centers')
    ax.scatter3D(point11[2], point11[0], point11[1], 'y.', label='Point1')
    break

plt.legend()
plt.show()
# plt.savefig("tmp/exampl.png")

NameError: name 'dl' is not defined

In [8]:
x = np.array([0.00000, 45.77594, -0.00000, -5.24599, 5.89959])
y = np.array([0.00000, 0.00000, 55.82716, 43.18748, 69.42272])

In [9]:
from src.utils.geometry import get_ellipse_from_points
centr, params, vectors = get_ellipse_from_points(x, y)

LinAlgError: Singular matrix

In [10]:
D = np.column_stack((x ** 2, x * y, y ** 2, x, y))

In [11]:
D

array([[   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ],
       [2095.43668288,    0.        ,    0.        ,   45.77594   ,
           0.        ],
       [   0.        ,   -0.        , 3116.67179367,   -0.        ,
          55.82716   ],
       [  27.52041108, -226.56108821, 1865.15842875,   -5.24599   ,
          43.18748   ],
       [  34.80516217,  409.56558468, 4819.5140522 ,    5.89959   ,
          69.42272   ]])